<a href="https://colab.research.google.com/github/sunyingjian/study-tensorflow/blob/master/%E8%87%AA%E5%AE%9A%E4%B9%89%E5%BE%AA%E7%8E%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#标准开场,引入tf2.0
%tensorflow_version 2.x
import tensorflow as tf
print('tensorflow:',format(tf.__version__))
import numpy as np
from tensorflow import keras
import glob
from tensorflow.keras import layers
import matplotlib.pyplot as plt
%matplotlib inline

tensorflow: 2.2.0-rc1


In [0]:
#引入minist数据集
(train_image,train_label),(test_image,test_label)=tf.keras.datasets.mnist.load_data()

In [3]:
#给图像增加维度
train_image=tf.expand_dims(train_image,-1)
train_image.shape

TensorShape([60000, 28, 28, 1])

In [0]:
#test图像增加维度
test_image=tf.expand_dims(test_image,-1)

In [0]:
#预处理，改变数据类型
train_image=tf.cast(train_image/255,tf.float32)
test_image=tf.cast(test_image/255,tf.float32)

In [0]:
#预处理，转换label的数据类型
train_label=tf.cast(train_label,tf.int64)
test_label=tf.cast(test_label,tf.int64)

In [0]:
#把它作为tf.data加载进来,建立dataset
dataset=tf.data.Dataset.from_tensor_slices((train_image,train_label))

In [8]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [0]:
test_dataset=tf.data.Dataset.from_tensor_slices((test_image,test_label))

In [0]:
#对数据进行乱序，这里对10000个数据进行乱序混淆
#批次的大小,用batch
dataset=dataset.shuffle(10000).batch(32)
#这里没有设置repeat
test_dataset=test_dataset.batch(32)

In [0]:
#建立模型，进行自定义训练,把整个网络写在list里
#这里input_shape如何让添加任意形状的图片呢，我们可以这样写参数(none,none,1)
model=tf.keras.Sequential([tf.keras.layers.Conv2D(16,[3,3],activation='relu',input_shape=(28,28,1)),
                           tf.keras.layers.Conv2D(32,[3,3],activation='relu'),
                           tf.keras.layers.GlobalAveragePooling2D(),
                           tf.keras.layers.Dense(10)
                           ])
#这里最后一层我没有写激活函数所以我们再后面的lossfunction里有写from_logitc=ture

In [12]:
#看一下model的可训练参数
model.trainable_variables

[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 16) dtype=float32, numpy=
 array([[[[-0.07251079,  0.14896826, -0.01542766, -0.1483107 ,
           -0.01370516,  0.10915162, -0.18296751,  0.1504984 ,
            0.07748096, -0.00580594,  0.06531979,  0.12766509,
            0.1606149 , -0.10996465, -0.18139958,  0.07155599]],
 
         [[-0.04529075, -0.13903813,  0.04226151, -0.19372435,
            0.19088109, -0.0483937 , -0.08575062, -0.12797624,
            0.04447636, -0.00594962,  0.0272837 ,  0.09850518,
            0.01890312,  0.19436453,  0.14159967,  0.13545142]],
 
         [[-0.05804266,  0.00544901,  0.04821381, -0.00765342,
           -0.14892723,  0.0902264 , -0.15722658,  0.03649324,
           -0.18054733, -0.1702545 , -0.19263107,  0.06331636,
            0.13785164, -0.19610728,  0.1170717 , -0.06297179]]],
 
 
        [[[-0.09805328,  0.05294393, -0.17177144,  0.15200232,
            0.09248696,  0.18757446, -0.1359687 , -0.05673753,
            0.04390535,  0.15

优化器

In [0]:
optimizer=tf.keras.optimizers.Adam()

自定义 损失函数

In [0]:
loss_func=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [0]:
#设计一个生成器
features,labels=next(iter(dataset))

In [16]:
features.shape

TensorShape([32, 28, 28, 1])

In [17]:
predictions=model(features)
predictions.shape

TensorShape([32, 10])

In [18]:
#哪个最大呢？
tf.argmax(predictions,axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([3, 3, 5, 3, 3, 3, 3, 3, 5, 5, 3, 5, 3, 3, 3, 5, 3, 3, 3, 5, 3, 3,
       3, 5, 3, 5, 3, 3, 3, 3, 3, 3])>

In [0]:
#开始训练
#定义一个loss函数
def loss(model,x,y):#x是实际的数据，y是实际的label
  y_=model(x)#y_是预测的label
  return loss_func(y,y_)
#这样的话 我们输入一个model和数据还有标签就会返回一个loss损失函数

##整合metrcis到自定义循环中

In [0]:
train_loss=tf.keras.metrics.Mean('train_loss')
#所以mean方法就是在求均值
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
#SparseCategoricalAcuracy就是在求均值
test_loss=tf.keras.metrics.Mean('test_loss')
test_accuracy=tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [0]:
#一步优化过程
def train_step(model,images,labels):
  with tf.GradientTape() as t:
    pred=model(images)#这里生成预测值
    loss_step=loss_func(labels,pred)
    #计算损失函数
    loss_step=loss(model,images,labels)
  grads=t.gradient(loss_step,model.trainable_variables)
  #优化grads 使梯度下降最快
  optimizer.apply_gradients(zip(grads,model.trainable_variables))
  train_loss(loss_step)
  train_accuracy(labels,pred)
  #这里就是应用这个梯度对可训练参数进行优化
  #以上就完成了对一个batch数据的优化

In [0]:
#建立test_step
def test_step(model,images,labels):
  pred=model(images)
  loss_step=loss_func(labels,pred)
  test_loss(loss_step)
  test_accuracy(labels,pred)

In [0]:
def train():#定义多层训练函数
  for epoch in range(10):
    #然后我们就可以对dataset进行迭代
    for (batch,(image,label)) in enumerate(dataset):
      #这里使用了enumerate枚举的方法
      train_step(model,image,label)
      #这样的话就进行了一次训练
    print('Epoch{} is finshed,loss is{},acc is {}'.format(epoch,
                                                          train_loss.result(),
                                                          train_accuracy.result()))
    for (batch,(image,label)) in enumerate(test_dataset):
      #这里使用了enumerate枚举的方法
      test_step(model,image,label)
    print('Epoch{} is finshed,test_loss is{},test_acc is {}'.format(epoch,
                                                          test_loss.result(),
                                                          test_accuracy.result()))
    #实现状态的重置，因为每个epoch是不一样的
    test_loss.reset_states()
    test_accuracy.reset_states()
    train_loss.reset_states()
    train_accuracy.reset_states()

In [24]:
train()

Epoch0 is finshed,loss is1.7392185926437378,acc is 0.34751665592193604
Epoch0 is finshed,test_loss is1.467941403388977,test_acc is 0.447299987077713
Epoch1 is finshed,loss is1.3740217685699463,acc is 0.5266333222389221
Epoch1 is finshed,test_loss is1.1911839246749878,test_acc is 0.6086000204086304
Epoch2 is finshed,loss is1.1189426183700562,acc is 0.6377833485603333
Epoch2 is finshed,test_loss is0.9631350636482239,test_acc is 0.7053999900817871
Epoch3 is finshed,loss is0.9279780983924866,acc is 0.7087000012397766
Epoch3 is finshed,test_loss is0.8045744299888611,test_acc is 0.7512000203132629
Epoch4 is finshed,loss is0.7997111082077026,acc is 0.7519333362579346
Epoch4 is finshed,test_loss is0.708358883857727,test_acc is 0.7820000052452087
Epoch5 is finshed,loss is0.7106226682662964,acc is 0.7828333377838135
Epoch5 is finshed,test_loss is0.6374936699867249,test_acc is 0.8072999715805054
Epoch6 is finshed,loss is0.6455931067466736,acc is 0.8026000261306763
Epoch6 is finshed,test_loss is0.

#tf.keras.metics 汇总计算模块

In [25]:
#初始化一个对象
m=tf.keras.metrics.Mean('acc')#mean方法就是一个计算均值的方法
#调用他
m(10)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [26]:
m(155)

<tf.Tensor: shape=(), dtype=float32, numpy=82.5>

所以这里为什么是82.5是因为他计算了155和10的均值

In [27]:
#返回他计算均值的结果
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=82.5>

In [28]:
m([30,40])
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=58.75>

In [0]:
#重置状态
m.reset_states()

In [30]:
m.result().numpy()

0.0

##汇总计算模块

In [31]:
#初始化一个对象
a=tf.keras.metrics.SparseCategoricalAccuracy('acc')
#为什么用这个SCA是因为我们前面的label是普通的编码不是独热编码
a(labels,model(features))
#这里的话就是在计算acc前面的是真的label的值，后面是预测的所以这样
#就能求出准确率，这样我们就可以添加到我们的自定义循环的代码中，这样就很方便了

<tf.Tensor: shape=(), dtype=float32, numpy=0.9375>